<a href="https://colab.research.google.com/github/ngwalker93/ADS-504-Final-Team-Project/blob/main/ADS_504FinalProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 504 Final Team Project
# Weight Lifting Exercises monitored with Inertial Measurement Units

### Team 6: Kiara Paz, Jordan Torres, and Nancy Walker

## Introduction
 This project aims to assess the application of machine learning methods and programming techniques.

## Dataset Discription
Weight Lifting Exercises monitored with Inertial Measurement Units: https://archive.ics.uci.edu/dataset/273/weight+lifting+exercises+monitored+with+inertial+measurement+units

The data set was sourced from the UC Irvine Machine Learning Repository. It contains observations of six healthy subjects that performed 5 variations of the bicep curl weight lifting exercise. A health care professional predicted one of the varaitions.

## Problem Definition


## Import Libraries and Dataset

In [1]:
# Import Libaries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Attach GitHub repository to retrieve dataset
!git clone https://github.com/ngwalker93/ADS-504-Final-Team-Project.git

Cloning into 'ADS-504-Final-Team-Project'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 3), reused 2 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 466.07 KiB | 10.84 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [4]:
# Check that repository was added
!ls ADS-504-Final-Team-Project

ADS_504FinalProj.ipynb
Example_WearableComputing_weight_lifting_exercises_biceps_curl_variations.csv
README.md


In [6]:
# Read .csv file
df = pd.read_csv('ADS-504-Final-Team-Project/Example_WearableComputing_weight_lifting_exercises_biceps_curl_variations.csv')
df.head()

EXAMPLE DATA: get the complete file at http://groupware.les.inf.puc-rio.br/har#weight_lifting_exercises
user_name raw_timestamp_part_1 raw_timestamp_part_2 cvtd_timestamp   new_window num_window roll_belt pitch_belt yaw_belt total_accel_belt kurtosis_roll_belt kurtosis_picth_belt kurtosis_yaw_belt skewness_roll_belt skewness_roll_belt skewness_yaw_belt max_roll_belt max_picth_belt max_yaw_belt min_roll_belt min_pitch_belt min_yaw_belt amplitude_roll_belt amplitude_pitch_belt amplitude_yaw_belt var_total_accel_belt avg_roll_belt stddev_roll_belt var_roll_belt avg_pitch_belt stddev_pitch_belt var_pitch_belt avg_yaw_belt stddev_yaw_belt var_yaw_belt gyros_belt_x gyros_belt_y gyros_belt_z accel_belt_x accel_belt_y accel_belt_z magnet_belt_x magnet_belt_y magnet_belt_z roll_arm pitch_arm yaw_arm total_accel_arm var_accel_arm avg_roll_arm stddev_roll_arm var_roll_arm avg_pitch_arm stddev_pitch_arm var_pitch_arm avg_yaw_arm stddev_yaw_arm var_yaw_arm gyros_arm_x gyros_arm_y gyros_arm_z accel_arm_x accel_arm_y accel_arm_z magnet_arm_x magnet_arm_y magnet_arm_z kurtosis_roll_arm kurtosis_picth_arm kurtosis_yaw_arm skewness_roll_arm skewness_pitch_arm skewness_yaw_arm max_roll_arm max_picth_arm max_yaw_arm min_roll_arm min_pitch_arm min_yaw_arm amplitude_roll_arm amplitude_pitch_arm amplitude_yaw_arm roll_dumbbell pitch_dumbbell yaw_dumbbell kurtosis_roll_dumbbell kurtosis_picth_dumbbell kurtosis_yaw_dumbbell skewness_roll_dumbbell skewness_pitch_dumbbell skewness_yaw_dumbbell max_roll_dumbbell max_picth_dumbbell max_yaw_dumbbell min_roll_dumbbell min_pitch_dumbbell min_yaw_dumbbell amplitude_roll_dumbbell amplitude_pitch_dumbbell amplitude_yaw_dumbbell total_accel_dumbbell var_accel_dumbbell avg_roll_dumbbell stddev_roll_dumbbell var_roll_dumbbell avg_pitch_dumbbell stddev_pitch_dumbbell var_pitch_dumbbell avg_yaw_dumbbell stddev_yaw_dumbbell var_yaw_dumbbell gyros_dumbbell_x gyros_dumbbell_y gyros_dumbbell_z accel_dumbbell_x accel_dumbbell_y accel_dumbbell_z magnet_dumbbell_x magnet_dumbbell_y magnet_dumbbell_z roll_forearm pitch_forearm yaw_forearm kurtosis_roll_forearm kurtosis_picth_forearm kurtosis_yaw_forearm skewness_roll_forearm skewness_pitch_forearm skewness_yaw_forearm max_roll_forearm max_picth_forearm max_yaw_forearm min_roll_forearm min_pitch_forearm min_yaw_forearm amplitude_roll_forearm amplitude_pitch_forearm amplitude_yaw_forearm total_accel_forearm var_accel_forearm avg_roll_forearm stddev_roll_forearm var_roll_forearm avg_pitch_forearm stddev_pitch_forearm var_pitch_forearm avg_yaw_forearm stddev_yaw_forearm var_yaw_forearm gyros_forearm_x gyros_forearm_y gyros_forearm_z accel_forearm_x accel_forearm_y accel_forearm_z magnet_forearm_x magnet_forearm_y magnet_forearm_z                                             classe                                                     
eurico    1322489729           34670                28/11/2011 14:15 no         1          3.7       41.6       -82.8    3                NaN                NaN                 NaN               NaN                NaN                NaN               NaN           NaN            NaN          NaN           NaN            NaN          NaN                 NaN                  NaN                NaN                  NaN           NaN              NaN           NaN            NaN               NaN            NaN          NaN             NaN          2.02         0.18         0.02         -3           -18          22           387           525           -267          132      -43.7     -53.6   38              NaN           NaN          NaN             NaN          NaN           NaN              NaN           NaN         NaN            NaN         2.65        -0.61       -0.02       143         30          -346        556          -205         -374         NaN               NaN                NaN              NaN               NaN                NaN              NaN          NaN           NaN         NaN          NaN           NaN         NaN              

## Exploratory Data Analysis (EDA)

## Pre-Processing

## Modeling Methods

## Validation

## Performance Metrics

## Modeling Results and Findings